## Initial work

### imports

In [ ]:
import numpy as np
import csv
import pandas as pd

from sklearn.model_selection import train_test_split

### data preporation

download data

In [ ]:
df = pd.read_csv("https://github.com/Roni42/just_some_public_data/blob/main/good_disk_new.csv?raw=true")
df['length_minutes'] = df['length_minutes'].map(lambda num: int(num[1:-1]))
df['year'] = df['year'].map(lambda num: int(num[1:-1]))
df.head(5)

fit transform

In [ ]:
from sklearn.preprocessing import StandardScaler
numerical_features = ['length_minutes', 'year', 'number_of_disks']
scaler = StandardScaler()
df[numerical_features] = scaler.fit_transform(df[numerical_features])
df.head(5)

show

In [ ]:
df.info()
df.describe()

separate data

In [ ]:
x = df[['number_of_disks', 'length_minutes', 'year']]
y = df['price'].values.tolist()

In [ ]:
print(x.shape, len(y))

In [ ]:
len(y)
x_train, x_temp, y_train, y_temp = train_test_split(x, y, shuffle=True, test_size=0.3, random_state=42)
x_valid, x_test, y_valid, y_test = train_test_split(x_temp, y_temp, shuffle=True, test_size=0.5, random_state=42)

Lib version

In [ ]:
neighbors_range = [i for i in range(1, 200)]

train_scores = []
valid_scores = []
test_scores = []

for n_neighbors in neighbors_range:
    model = KNeighborsClassifier(n_neighbors=n_neighbors)
    model.fit(x_train, y_train)

    # y_train_pred = model.predict(x_train)
    # train_accuracy = accuracy_score(y_train, y_train_pred)
    # train_scores.append(train_accuracy)

    y_valid_pred = model.predict(x_valid)
    valid_accuracy = accuracy_score(y_valid, y_valid_pred)
    valid_scores.append(valid_accuracy)

    y_test_pred = model.predict(x_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_scores.append(test_accuracy)

plt.figure(figsize=(10, 6))
# plt.plot(neighbors_range, train_scores, label='Train Accuracy')
plt.plot(neighbors_range, valid_scores, label='Valid (Test) Accuracy')
plt.plot(neighbors_range, test_scores, label='Real Test Accuracy')

plt.xlabel('Number of Neighbors')
plt.ylabel('Accuracy')
plt.title('Impact of Number of Neighbors on Model Accuracy')

plt.legend()
plt.show()

In [ ]:
model = KNeighborsClassifier(n_neighbors=150)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
plt.plot(y_test, color="red")
plt.plot(y_pred, color="blue")

In [ ]:
x.shape, len(y)

In [ ]:
x_train, x_temp, y_train, y_temp = train_test_split(x, y, shuffle=True, test_size=0.3, random_state=42)
x_valid, x_test, y_valid, y_test = train_test_split(x_temp, y_temp, shuffle=True, test_size=0.5, random_state=42)

## My realisation

## Lib realisation

## Tests